# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457774 , -1.8541752 , -5.1695833 ,
         7.6670885 ,  2.7118318 ,  8.495609  ,  1.7038486 ,  1.1884269 ],
       [ 4.6476874 ,  8.37788   , -9.070581  ,  9.459332  ,  8.450422  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.4838412 ],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.8169358 ],
       [-4.2710767 ,  5.5611653 , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876224 , -5.1257744 ,  9.694357  ],
       [ 5.583742  , -4.1515303 ,  4.369667  , -3.0020502 ,  3.638897  ,
        -4.341912  , -3.3187115 ,  6.503671  , -6.865036  , -1.0266497 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:44750 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.8098755 ,  8.422671  , -9.239025  ,  9.379142  ,  8.499884  ,
        -1.0592817 ,  3.343786  , -7.80261   , -0.5946333 ,  0.26447597,
         5.507395  , -4.1069794 ,  4.289077  , -2.8172052 ,  3.6150148 ,
        -4.1612997 , -3.6209636 ,  6.2185307 , -6.946049  , -1.0828307 ,
        -5.8267713 ,  2.225876  , -3.8601213 , -1.6974076 , -5.3134165 ,
         7.579578  ,  2.9187474 ,  8.540423  ,  1.5523204 ,  1.0841808 ],
       [-2.894186  ,  4.474189  , -4.447567  ,  2.382099  ,  1.7478833 ,
        -2.504626  , -5.208331  , -1.6937685 , -8.134755  ,  2.6468296 ,
        -4.316363  ,  5.5655403 , -5.732199  , -1.7384951 , -9.344656  ,
         0.7084658 ,  4.4358377 , -2.9009006 , -4.948639  ,  9.695302  ,
         8.366523  , -6.2474537 , -6.3494735 ,  1.9546981 ,  4.1576157 ,
        -9.167905  ,  4.607067  ,  8.788585  ,  6.8644223 ,  2.2319875 ],
       [-4.665716  , -9.558953  ,  6.657229  ,  4.4401317 ,  2.17303   ,
         2.5904028 ,  0.58000994,  6.255035  , -8